## Linking in Spark


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/examples/spark/deduplicate_1k_synthetic.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev
# !pip install pyspark

In [2]:
from splink.spark.jar_location import similarity_jar_location

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
# This parallelism setting is only suitable for a small toy example
conf.set("spark.driver.memory", "12g")
conf.set("spark.default.parallelism", "16")


# Add custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
path = similarity_jar_location()
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

24/03/13 12:30:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# Disable warnings for pyspark - you don't need to include this
import warnings

spark.sparkContext.setLogLevel("ERROR")
warnings.simplefilter("ignore", UserWarning)

In [4]:
from splink import splink_datasets

pandas_df = splink_datasets.fake_1000

df = spark.createDataFrame(pandas_df)

In [5]:
import splink.comparison_library as cl
import splink.comparison_template_library as ctl

from splink import Linker, SparkAPI, block_on, SettingsCreator

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        ctl.NameComparison("first_name"),
        ctl.NameComparison("surname"),
        # ctl.date_comparison("dob", cast_strings_to_date=True),
        # TOD=Fix date comparison
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            datetime_metrics=["month", "year", "year"],
            datetime_thresholds=[1, 1, 10],
            datetime_format="%Y%m%d",
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        ctl.EmailComparison("email", include_username_fuzzy_level=False),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        "l.surname = r.surname",  # alternatively, you can write BRs in their SQL form
    ],
    retain_intermediate_calculation_columns=True,
    em_convergence=0.01,
)

In [6]:
linker = Linker(df, settings, database_api=SparkAPI(spark_session=spark))
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

Probability two random records match is estimated to be  0.0806.
This means that amongst all possible pairwise record comparisons, one in 12.41 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 40,246.67 matching pairs


In [7]:
linker.estimate_u_using_random_sampling(max_pairs=5e5)

----- Estimating u probabilities using random sampling -----



Estimated u probabilities using random sampling



Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [8]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_fname_sname = (
    linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)

training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname


Iteration 1: Largest change in params was -0.697 in probability_two_random_records_match


Iteration 2: Largest change in params was 0.0565 in the m_probability of email, level `All other comparisons`


Iteration 3: Largest change in params was 0.019 in the m_probability of email, level `All other comparisons`


...

Iteration 3: Largest change in params was 0.0431 in probability_two_random_records_match


Iteration 4: Largest change in params was 0.0178 in probability_two_random_records_match


Iteration 5: Largest change in params was 0.00899 in probability_two_random_records_match



EM converged after 5 iterations



Your model is fully trained. All comparisons have at least one estimate for their m and u values


In [9]:
results = linker.predict(threshold_match_probability=0.9)

In [10]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,...,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,match_key
0,16.439400,0.999989,533,536,Freddie,Freddie,3,11.44555,Gregory,Grerrogy,...,0,0.173,0.187,0.624788,1.000000,freddiegregory71@jensen.info,freddiegregory71@jensen.info,3,8.472959,0
1,10.030650,0.999045,838,841,Charlotte,Charlotte,3,11.44555,Harper,Harper,...,0,0.187,0.012,0.624788,1.000000,charpe@r@milleb.riz,charper@miller.biz,1,252.763068,0
2,7.476405,0.994416,365,369,Samuel,Samuel,3,11.44555,Campbell,Campbell,...,0,0.001,0.018,0.624788,1.000000,samuelcampbell35@hebert.com,samuelcampbell35@hebert.com,3,8.472959,0
3,25.105978,1.000000,606,608,Amelia,Amelia,3,11.44555,Porter,Porrret,...,1,0.014,0.014,5.890233,5.089947,ameiiap@nlcholson.orrg,ameliap@nicholson.org,1,252.763068,0
4,8.192765,0.996594,35,416,NaN,NaN,3,11.44555,Bron,Brown,...,0,0.009,0.187,0.624788,1.000000,NaN,NaN,3,8.472959,0
